# Execution Trace Demo

In [1]:
from hta.trace_analysis import TraceAnalysis

In [2]:
from hta.common import execution_trace

In [3]:
import pandas as pd
import numpy as np

## Load Kineto traces and Execution Trace

__<font color='red'>
Note: To run the notebook, ensure that the path to the HolisticTraceAnalysis repo is set appropriately in the `trace_dir` variable below.
</font>__

In [4]:
trace_prefix = "~/HolisticTraceAnalysis"
trace_prefix = "~/Work/hta/HolisticTraceAnalysis"
trace_dir = f"{trace_prefix}/tests/data/execution_trace/"
analyzer = TraceAnalysis(trace_dir=trace_dir)

2023-07-14 17:38:51,593 - hta - trace.py:L390 - INFO - /Users/bcoutinho/Work/hta/HolisticTraceAnalysis/tests/data/execution_trace
2023-07-14 17:38:51,599 - hta - trace_file.py:L61 - ERROR - If the trace file does not have the rank specified in it, then add the following snippet key to the json files to use HTA; "distributedInfo": {"rank": 0}. If there are multiple traces files, then each file should have a unique rank value.
2023-07-14 17:38:51,599 - hta - trace_file.py:L94 - INFO - Rank to trace file map:
{0: '/Users/bcoutinho/Work/hta/HolisticTraceAnalysis/tests/data/execution_trace/benchmark_simple_add_trace.json.gz'}
2023-07-14 17:38:51,600 - hta - trace.py:L536 - INFO - ranks=[0]
2023-07-14 17:38:51,601 - hta - trace.py:L118 - INFO - Parsed /Users/bcoutinho/Work/hta/HolisticTraceAnalysis/tests/data/execution_trace/benchmark_simple_add_trace.json.gz time = 0.00 seconds 
2023-07-14 17:38:51,621 - hta - trace.py:L672 - WARNING - ProfilerStep not found in the trace. The analysis resul

In [5]:
et = execution_trace.load_execution_trace(trace_dir + "benchmark_simple_add_et.json.gz")

## Correlate Execution Trace and Kineto/PyTorch Profiler Trace

In [6]:
execution_trace.correlate_execution_trace(analyzer.t, 0, et)

2023-07-14 17:38:51,631 - root - execution_trace.py:L53 - INFO - Trace and ET have overlap = True
2023-07-14 17:38:51,632 - root - execution_trace.py:L54 - INFO - Trace rf_ids (1, 83),ET rf_ids (1, 36)


In [7]:
trace_df = analyzer.t.get_trace(0)

In [8]:
trace_df.head()

,index,cat,name,pid,tid,ts,dur,Trace iteration,correlation,stream,External id,memory_bw_gbps,index_correlation,iteration,et_node
0,0,23,26,563677,563677,0,19814157,-1,-1,-1,1,-1,-1,-1,3.0
1,1,27,16,563677,563677,2006,19583658,-1,-1,-1,2,-1,-1,-1,4.0
2,2,27,10,563677,563677,2047,19583280,-1,-1,-1,3,-1,-1,-1,5.0
3,3,27,9,563677,563677,19585454,189,-1,-1,-1,4,-1,-1,-1,8.0
4,4,27,16,563677,563677,19585799,94,-1,-1,-1,5,-1,-1,-1,9.0


In [9]:
execution_trace.add_et_column(trace_df, et, 'op_schema')
execution_trace.add_et_column(trace_df, et, 'input_shapes')
execution_trace.add_et_column(trace_df, et, 'input_types')
execution_trace.add_et_column(trace_df, et, 'output_shapes')
execution_trace.add_et_column(trace_df, et, 'output_types')

In [10]:
trace_df.head()

,index,cat,name,pid,tid,ts,dur,Trace iteration,correlation,stream,External id,memory_bw_gbps,index_correlation,iteration,et_node,op_schema,input_shapes,input_types,output_shapes,output_types
0,0,23,26,563677,563677,0,19814157,-1,-1,-1,1,-1,-1,-1,3.0,,[],[],[],[]
1,1,27,16,563677,563677,2006,19583658,-1,-1,-1,2,-1,-1,-1,4.0,"aten::rand(SymInt[] size, *, ScalarType? dtype...","[[[], []], [], [], [], []]","[GenericList[Int,Int], Int, None, Device, Bool]","[[256, 256]]",[Tensor(float)]
2,2,27,10,563677,563677,2047,19583280,-1,-1,-1,3,-1,-1,-1,5.0,"aten::empty.memory_format(SymInt[] size, *, Sc...","[[[], []], [], [], [], [], []]","[GenericList[Int,Int], Int, None, Device, Bool...","[[256, 256]]",[Tensor(float)]
3,3,27,9,563677,563677,19585454,189,-1,-1,-1,4,-1,-1,-1,8.0,"aten::uniform_(Tensor(a!) self, float from=0.,...","[[256, 256], [], [], []]","[Tensor(float), Double, Double, None]","[[256, 256]]",[Tensor(float)]
4,4,27,16,563677,563677,19585799,94,-1,-1,-1,5,-1,-1,-1,9.0,"aten::rand(SymInt[] size, *, ScalarType? dtype...","[[[], []], [], [], [], []]","[GenericList[Int,Int], Int, None, Device, Bool]","[[256, 256]]",[Tensor(float)]
